In [1]:
import numpy as np
import numpy.linalg as la
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

%matplotlib inline

2025-09-24 12:53:03.773832: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-24 12:53:03.794410: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-24 12:53:04.506885: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-09-24 12:53:05.917149: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation or

# RNNs

The base Tensorflow Keras RNN is an Elman RNN. All that is needed is to code a Jordan and Multi-RNN.

In [ ]:
class JordanRNNCell(tf.keras.layers.AbstractRNNCell):

    def __init__(self, units, output_dim, activation='tanh', output_activation=None, kernel_init='glorot_uniform', recurrent_init='glorot_uniform', bias_init='zeros', **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.output_dim = output_dim
        self.activation = tf.keras.activations.get(activation)
        self.output_activation = tf.keras.activations.get(output_activation)
        self.kernel_init = tf.keras.initializers.get(kernel_init)
        self.recurrent_init = tf.keras.initializers.get(recurrent_init)
        self.bias_init = tf.keras.initializers.get(bias_init)

        self.state_size = output_dim
        self.output_size = output_dim

    def build(self, input_shape):
        input_dim = input_shape[-1]

        self.Wx = self.add_weight(shape=(input_dim, self.units), initializer=self.kernel_init, name='Wx')
        self.Wy = self.add_weight(shape=(self.output_dim, self.units), initializer=self.recurrent_init, name='Wy')
        self.bh = self.add_weight(shape=(self.units, ), initializer=self.bias_init, name='bh')

        self.Wo = self.add_weight(shape=(self.units, self.output_dim), initializer=self.kernel_init, name='Wo')
        self.bo = self.add_weight(shape=(self.output_dim, ), initializer=self.bias_init, name='bo')

        super().build(input_shape)

    def call(self, inputs, states, training=None):
        y_prev = states[0]

        h_t = tf.matmul(inputs, self.Wx) + tf.matmul(y_prev, self.Wy) + self.bh
        h_t = self.activation(h_t)

        y_t = tf.matmul(h_t, self.Wo) + self.bo

        if self.output_activation is not None:
            y_t = self.output_activation(y_t)
        
        return y_t, [y_t]
    
    def get_config(self):
        base = super().get_config()
        base.update({
            "units": self.units,
            "output_dim": self.output_dim,
            "activation": tf.keras.activations.serialize(self.activation),
            "output_activation": tf.keras.activations.serialize(self.output_activation),
            "kernel_initializer": tf.keras.initializers.serialize(self.kernel_init),
            "recurrent_initializer": tf.keras.initializers.serialize(self.recurrent_init),
            "bias_initializer": tf.keras.initializers.serialize(self.bias_init),
        })
        return base


def JordanRNN(units, output_dim, return_sequences=False, **kwargs):
    cell = JordanRNNCell(units=units, output_dim=output_dim, **kwargs)
    return tf.keras.layers.RNN(cell, return_sequences=return_sequences)


def build_jordan_model(input_dim, units=64, output_dim=1, return_sequences=False, output_activation=None):
    inp = tf.keras.Input(shape=(None, input_dim))
    x = JordanRNN(units=units, output_dim=output_dim, return_sequences=return_sequences, activation='tanh', output_activation=output_activation)(inp)
    model = tf.keras.Model(inp, x)
    return model

In [ ]:
class MultiRNNCell(tf.keras.layers.AbstractRNNCell):

    def __init__(self, units, output_dim, activation='tanh', output_activation=None, kernel_init='glorot_uniform', recurrent_init='glorot_uniform', bias_init='zeros', **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.output_dim = output_dim
        self.activation = tf.keras.activations.get(activation)
        self.output_activation = tf.keras.activations.get(output_activation)
        self.kernel_init = tf.keras.initializers.get(kernel_init)
        self.recurrent_init = tf.keras.initializers.get(recurrent_init)
        self.bias_init = tf.keras.initializers.get(bias_init)

        self.state_size = [units, output_dim]
        self.output_size = output_dim
        self.supports_masking = True

    def build(self, input_shape):
        input_dim = input_shape[-1]

        self.Wx = self.add_weight(shape=(input_dim, self.units), initializer=self.kernel_init, name='Wx')
        self.Wh = self.add_weight(shape=(self.units, self.units), initializer=self.recurrent_init, name='Wh')
        self.Wy = self.add_weight(shape=(self.output_dim, self.units), initializer=self.recurrent_init, name='Wy')
        self.bh = self.add_weight(shape=(self.units, ), initializer=self.bias_init, name='bh')

        self.Wo = self.add_weight(shape=(self.units, self.output_dim), initializer=self.kernel_init, name='Wo')
        self.bo = self.add_weight(shape=(self.output_dim, ), initializer=self.bias_init, name='bo')

        super().build(input_shape)

    def call(self, inputs, states, training=None):
        h_prev, y_prev = states

        # takes both
        h_t = tf.matmul(inputs, self.Wx) + tf.matmul(h_prev, self.Wh) + tf.matmul(y_prev, self.Wy) + self.bh
        h_t = self.activation(h_t)

        y_t = tf.matmul(h_t, self.Wo) + self.bo

        if self.output_activation is not None:
            y_t = self.output_activation(y_t)
        
        return y_t, [h_t, y_t]
    
    def get_config(self):
        base = super().get_config()
        base.update({
            "units": self.units,
            "output_dim": self.output_dim,
            "activation": tf.keras.activations.serialize(self.activation),
            "output_activation": tf.keras.activations.serialize(self.output_activation),
            "kernel_initializer": tf.keras.initializers.serialize(self.kernel_init),
            "recurrent_initializer": tf.keras.initializers.serialize(self.recurrent_init),
            "bias_initializer": tf.keras.initializers.serialize(self.bias_init),
        })
        return base

    
def MultiRNN(units, output_dim, return_sequences=False, **kwargs):
    cell = MultiRNNCell(units=units, output_dim=output_dim, **kwargs)
    return tf.keras.layers.RNN(cell, return_sequences=return_sequences)


def build_multi_model(input_dim, units=64, output_dim=1, return_sequences=False, output_activation=None):
    inp = tf.keras.Input(shape=(None, input_dim))
    x = MultiRNN(units=units, output_dim=output_dim, return_sequences=return_sequences, activation='tanh', output_activation=output_activation)(inp)
    model = tf.keras.Model(inp, x)
    return model